In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dist
import numpy as np
import tqdm
from copy import deepcopy
import collections

In [2]:
class ReplayBuffer():
    def __init__(self, maxlen):
        self.buffer = collections.deque(maxlen=maxlen)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        ind = np.random.choice(len(self.buffer), n, replace=False)
        s, a, r, d, ns = zip(*[self.buffer[i] for i in ind])
        return np.array(s), np.array(a), np.array(r), \
            np.array(d), np.array(ns)
    
    def size(self):
        return len(self.buffer)

In [3]:
class Actor(nn.Module):
    def __init__(self, input_dim=3, hid_dim=128, nlayers=2, out_dim=1):
        super(Actor, self).__init__()
        self.model = []
        self.out_dim = out_dim
        inp = input_dim
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model = nn.Sequential(*self.model)
        self.mu = nn.Linear(inp, out_dim)
        self.std = nn.Linear(inp, out_dim)

    def forward(self, x):
        out = self.model(x)
        mu = self.mu(out)
        std = F.softplus(self.std(out))
        dist = torch.distributions.Normal(mu, std)

        action = dist.rsample()
        log_prob = dist.log_prob(action)
        real_action = torch.tanh(action)
        real_log_prob = log_prob - torch.log(1-real_action.pow(2)+1e-7)
        return real_action, real_log_prob


class Critic(nn.Module):
    def __init__(self, s_dim=3, a_dim=1, hid_dim=64, nlayers=1, out_dim=1):
        super(Critic, self).__init__()
        self.linear_s = nn.Sequential(
            nn.Linear(s_dim, hid_dim),
            nn.ReLU()
        )
        self.linear_a = nn.Sequential(
            nn.Linear(a_dim, hid_dim),
            nn.ReLU()
        )
        
        self.model = []
        self.out_dim = out_dim
        inp = hid_dim * 2
        for i in range(nlayers):
            self.model.append(nn.Linear(inp, hid_dim))
            self.model.append(nn.ReLU())
            inp = hid_dim
        self.model.append(nn.Linear(inp, out_dim))
        self.model = nn.Sequential(*self.model)

    def forward(self, s, a):
        inp_s = self.linear_s(s)
        inp_a = self.linear_a(a)
        # 将s和a的输入拼接在一起再计算Q(s, a)
        inp = torch.cat([inp_s, inp_a], dim=1)
        return self.model(inp)

In [4]:
class SAC:
    def __init__(self, mu, mu_optim, q1, q1_optim, q2, q2_optim, gamma, tau, alpha=0.2):
        self.mu = mu
        self.mu_optim = mu_optim
        
        self.q1 = q1
        self.q1_target = deepcopy(q1)
        self.q1_target.eval()
        self.q1_optim = q1_optim
        
        self.q2 = q2
        self.q2_target = deepcopy(q2)
        self.q2_target.eval()
        self.q2_optim = q2_optim
        
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha
    
    def soft_update(self):
        # soft update 计算方式
        for p_t, p in zip(self.q1_target.parameters(), self.q1.parameters()):
            p_t.data.copy_(p_t.data * (1 - self.tau) + p.data * self.tau)
        
        for p_t, p in zip(self.q2_target.parameters(), self.q2.parameters()):
            p_t.data.copy_(p_t.data * (1 - self.tau) + p.data * self.tau)
    
    def calc_target(self, batch):
        s, a, ns, d, r = batch
        with torch.no_grad():
            na, log_prob = self.mu(ns)
            entropy = -self.alpha * log_prob
            q1_val, q2_val = self.q1_target(ns, na), self.q2_target(ns, na)
            min_q = torch.min(torch.cat([q1_val, q2_val], dim=1), 1, keepdim=True)[0]
            target = r + self.gamma * d * (min_q + entropy)
        return target
    
    def learn(self, buffer, batch_size):
        # 采样数据
        s, a, r, d, ns = buffer.sample(batch_size)
        s = torch.from_numpy(s).float()
        a = torch.from_numpy(a).float()[:, None]
        ns = torch.from_numpy(ns).float()
        d = torch.from_numpy(d).float()[:, None]
        r = torch.from_numpy(r).float()[:, None]
        
        self.mu.train()
        self.q1.train()
        self.q2.train()

        # 计算Q函数loss
        target = self.calc_target((s, a, ns, d, r))
        q1_loss = F.mse_loss(self.q1(s, a), target.detach())
        q2_loss = F.mse_loss(self.q2(s, a), target.detach())

        self.q1_optim.zero_grad()
        q1_loss.backward()
        self.q1_optim.step()
        
        self.q2_optim.zero_grad()
        q2_loss.backward()
        self.q2_optim.step()
        
        a, log_prob = self.mu(s)
        entropy = -self.alpha * log_prob
      
        q1_val, q2_val = self.q1(s, a), self.q2(s, a)
        min_q = torch.min(torch.cat([q1_val, q2_val], dim=1), 1, keepdim=True)[0]
        mu_loss = (-min_q - entropy).mean()
                   
        self.mu_optim.zero_grad()
        mu_loss.backward()
        self.mu_optim.step()
        
        
    def sample_action(self, state):
        self.mu.eval()
        state = torch.from_numpy(state)
        with torch.no_grad():
            action = self.mu(state)[0].item()
        return action

In [5]:
class Args:
    gamma = 0.99
    batch_size = 32
    tau = 5e-3
    sigma = 0.1
    max_episodes = 2500

In [6]:
args = Args

In [ ]:
all_score_list = []
for i in range(10):
    env = gym.make('Pendulum-v1')
    n_obs = env.observation_space.shape[0]
    buffer = ReplayBuffer(maxlen=50000)
    gamma = 0.98
    batch_size = 32
    tau = 5e-3
    alpha = 0.04
    max_episodes = 2500

    actor = Actor(n_obs, 128, 2, 1)
    actor_optim = torch.optim.Adam(actor.parameters(), lr=5e-4)
    critic1 = Critic(n_obs, 1, 64, 1, 1)
    critic1_optim = torch.optim.Adam(critic1.parameters(), lr=1e-3)
    critic2 = Critic(n_obs, 1, 64, 1, 1)
    critic2_optim = torch.optim.Adam(critic2.parameters(), lr=1e-3)
    policy = SAC(actor, actor_optim, critic1, critic1_optim, critic2, critic2_optim, gamma, tau, alpha)

    score_list = []
    for i in tqdm.tqdm(range(max_episodes)):
        s = env.reset()
        score = 0
        done = False
        while not done:
            a = policy.sample_action(s)
            next_s, r, done, info = env.step([a*2.0])
            d = 0.0 if done else 1.0
            buffer.put((s, a, r/100, d, next_s))
            s = next_s

            score += r

        if buffer.size() > 2000:
            for k in range(10):
                policy.learn(buffer, batch_size)
                policy.soft_update()

        score_list.append(score)
        
    all_score_list.append(score_list)